In [ ]:
import random
import math
from abc import ABC, abstractmethod

# Classe para representar uma rainha
class Rainha:
    def __init__(self, linha, coluna):
        self.linha = linha
        self.coluna = coluna

    def __eq__(self, other):
        """Sobrescreve o método de igualdade para comparar rainhas."""
        return self.linha == other.linha and self.coluna == other.coluna

    def __str__(self):
        return f"Rainha(linha={self.linha}, coluna={self.coluna})"

# Classe para representar o tabuleiro
class Tabuleiro:
    def __init__(self, linhas, colunas):
        self.rainhas = []
        self.linhas = linhas
        self.colunas = colunas

    def start(self):
        for coluna in range(self.colunas):
            linha = random.randint(0, self.linhas - 1)
            self.adicionar_rainha(Rainha(linha, coluna))

    def adicionar_rainha(self, rainha):
        self.rainhas.append(rainha)

    def remover_rainha(self, rainha):
        self.rainhas.remove(rainha)

    def calcular_conflitos(self):
        conflitos = 0
        for i in range(self.linhas):
            for j in range(i + 1, self.colunas):
                r1, r2 = self.rainhas[i], self.rainhas[j]
                if r1.linha == r2.linha or abs(r1.linha - r2.linha) == abs(r1.coluna - r2.coluna):
                    conflitos += 1
        return conflitos

    def clonar(self):
        novo_tabuleiro = Tabuleiro(self.linhas, self.colunas)
        novo_tabuleiro.rainhas = [Rainha(r.linha, r.coluna) for r in self.rainhas]
        return novo_tabuleiro

    def __str__(self):
        return "\n".join(str(rainha) for rainha in self.rainhas)

# Interface para algoritmos de resolução
class AlgoritmoResolucao(ABC):
    @abstractmethod
    def resolver(self, tabuleiro):
        pass

# Implementação do algoritmo de têmpera simulada
class TemperaSimulada(AlgoritmoResolucao):
    def __init__(self, temperatura_inicial=1000, taxa_resfriamento=0.95, iteracoes_por_temperatura=100):
        self.temperatura_inicial = temperatura_inicial
        self.taxa_resfriamento = taxa_resfriamento
        self.iteracoes_por_temperatura = iteracoes_por_temperatura

    def resolver(self, tabuleiro):
        temperatura = self.temperatura_inicial
        tabuleiro_atual = tabuleiro.clonar()
        conflitos_atual = tabuleiro_atual.calcular_conflitos()

        while temperatura > 1 and conflitos_atual > 0:
            for _ in range(self.iteracoes_por_temperatura):
                # Gera um vizinho aleatório
                novo_tabuleiro = self._gerar_vizinho(tabuleiro_atual)
                novos_conflitos = novo_tabuleiro.calcular_conflitos()

                # Calcula a diferença de conflitos
                delta_conflitos = novos_conflitos - conflitos_atual

                # Aceita a nova solução se for melhor ou com uma probabilidade baseada na temperatura
                if delta_conflitos < 0 or random.random() < math.exp(-delta_conflitos / temperatura):
                    tabuleiro_atual = novo_tabuleiro
                    conflitos_atual = novos_conflitos

            # Resfria a temperatura
            temperatura *= self.taxa_resfriamento

        return tabuleiro_atual

    def _gerar_vizinho(self, tabuleiro):
        """Gera um vizinho aleatório movendo uma rainha para uma nova posição na mesma coluna."""
        novo_tabuleiro = tabuleiro.clonar()
        coluna = random.randint(0, tabuleiro.colunas)
        rainha_atual = next((r for r in novo_tabuleiro.rainhas if r.coluna == coluna), None)
        if rainha_atual:
            novo_tabuleiro.remover_rainha(rainha_atual)
            nova_linha = random.randint(0, tabuleiro.linhas)
            novo_tabuleiro.adicionar_rainha(Rainha(nova_linha, coluna))
        return novo_tabuleiro

# Classe para visualizar o tabuleiro
class VisualizadorTabuleiro:
    @staticmethod
    def mostrar(tabuleiro):
        print("Conflitos: ", tabuleiro.calcular_conflitos())

        print("Solução encontrada:")
        for linha in range(tabuleiro.linhas):
            print(" ".join("Q" if any(r.linha == linha and r.coluna == coluna for r in tabuleiro.rainhas) else "." for coluna in range(tabuleiro.colunas)))



In [50]:
# Inicializa o tabuleiro com 8 rainhas em posições aleatórias
tabuleiro = Tabuleiro(8, 8)
tabuleiro.start()

algoritmo = TemperaSimulada(temperatura_inicial=1000, taxa_resfriamento=0.95, iteracoes_por_temperatura=100)
solucao = algoritmo.resolver(tabuleiro)

VisualizadorTabuleiro.mostrar(solucao)

Conflitos:  0
Solução encontrada:
. . . . . Q . .
. . . Q . . . .
. Q . . . . . .
. . . . . . . Q
. . . . Q . . .
. . Q . . . . .
Q . . . . . . .
. . . . . . . .


In [75]:
tabuleiro = Tabuleiro(15, 15)
tabuleiro.start()

algoritmo = TemperaSimulada(temperatura_inicial=1000, taxa_resfriamento=0.95, iteracoes_por_temperatura=100)
solucao = algoritmo.resolver(tabuleiro)

VisualizadorTabuleiro.mostrar(solucao)

Conflitos:  5
Solução encontrada:
. . . . . . . . . . . . . . .
. . Q . . . . . . . . . Q . .
. . . . . . . . . . Q . . . .
. . . Q . . . . . . . . . . .
. . . . . . . . . . . . . . Q
. . . . . . Q . . . . . . . .
. Q . . . . . . . Q . . . . .
. . . . . . . . . . . . . . .
. . . . . . . . Q . . . . . .
. . . . . . . . . . . . . . .
. . . . . . . . . . . . . . .
. . . . . . . . . . . . . Q .
. . . . . Q . . . . . Q . . .
. . . . . . . Q . . . . . . .
Q . . . . . . . . . . . . . .
